In [1]:
import sys
import os
sys.path.append(os.path.abspath('../../'))

In [2]:
import pandas as pd

from mcda.electre.concordance import concordance, concordance_marginal, concordance_comprehensive
from mcda.core.scales import QuantitativeScale, PreferenceDirection
from mcda.core.functions import Threshold

1. Create dataframes

In [3]:
alternative_names = ['fiat', 'vwPolo' , 'nissan' , 'toyota' , 'suzuki', 'ford' ]
criteria_names = ['C1', 'C2', 'C3']
profile_names = ['P1', 'P2', 'P3']

In [4]:
fiat = [100, 1000, 6.0]
vwPolo = [150, 5000, 5.7]
nissan = [110, 4000, 6.8]
toyota = [170, 3000, 5.5]
suzuki = [140, 2000, 7.8]
ford = [120, 6000, 8.0]
cars = pd.DataFrame(
    [fiat, vwPolo, nissan, toyota, suzuki, ford],
    index=alternative_names,
    columns=criteria_names,
)
cars

,C1,C2,C3
fiat,100,1000,6.0
vwPolo,150,5000,5.7
nissan,110,4000,6.8
toyota,170,3000,5.5
suzuki,140,2000,7.8
ford,120,6000,8.0


In [5]:
profile1 = [113, 1200, 7.0]
profile2 = [180, 3500, 6.7]
profile3 = [199, 3400, 8.8]
profiles = pd.DataFrame(
    [profile1, profile2, profile3],
    index=profile_names,
    columns=criteria_names
) 
profiles

,C1,C2,C3
P1,113,1200,7.0
P2,180,3500,6.7
P3,199,3400,8.8


2. Create series with thresholds and specify criteria scales and weights

In [6]:
scales = pd.Series([
    QuantitativeScale(100, 500),
    QuantitativeScale(1000, 6000, PreferenceDirection.MIN),
    QuantitativeScale(0.0, 15.0, PreferenceDirection.MIN)
], index=criteria_names)
scales

C1      Scale [100, 500], max direction
C2    Scale [1000, 6000], min direction
C3     Scale [0.0, 15.0], min direction
dtype: object

In [7]:
weights = pd.Series([2, 3, 5], index=criteria_names)
weights

C1    2
C2    3
C3    5
dtype: int64

In [8]:
indifference = pd.Series([
    Threshold(0.1, 10),
    Threshold(0, 500),
    Threshold(0, 0.1)
], index=criteria_names)
indifference

C1    Threshold: alpha=0.1, beta=10
C2     Threshold: alpha=0, beta=500
C3     Threshold: alpha=0, beta=0.1
dtype: object

In [9]:
preference = pd.Series([
    Threshold(0.3, 20),
    Threshold(0.4, 500),
    Threshold(0.1, 0.7)
], index=criteria_names)
preference

C1     Threshold: alpha=0.3, beta=20
C2    Threshold: alpha=0.4, beta=500
C3    Threshold: alpha=0.1, beta=0.7
dtype: object

3. Let's begin with some marginal concordance calculations :)

Marginal concordance index between fiat and suzuki on the C2 criterion:

In [10]:
concordance_marginal(
    cars['C2']['fiat'],
    cars['C2']['suzuki'],
    scales['C2'],
    indifference['C2'],
    preference['C2']
)

1.0

4. Short example with comprehensive index calculations

Again, between fiat and suzuki:

In [11]:
concordance_comprehensive(
    cars.loc['fiat'],
    cars.loc['suzuki'],
    scales,
    weights,
    indifference,
    preference
)

0.8666666666666666

5. Concordance table between alternatives and alternatives

In [12]:
concordance(
    cars,
    scales,
    weights,
    indifference,
    preference
)

,fiat,vwPolo,nissan,toyota,suzuki,ford
fiat,1.000000,0.716667,1.000000,0.633333,0.866667,1.0
vwPolo,0.700000,1.000000,0.925000,0.732265,0.700000,1.0
nissan,0.426563,0.490625,1.000000,0.237500,0.662500,1.0
toyota,0.700000,1.000000,1.000000,1.000000,0.875000,1.0
suzuki,0.312500,0.500000,0.673913,0.468421,1.000000,1.0
ford,0.200000,0.390441,0.419643,0.035294,0.664286,1.0


6. Concordance table between alternatives and profiles

In [13]:
alt_prof, prof_alt = concordance(
    cars,
    scales,
    weights,
    indifference,
    preference,
    profiles,
)

In [14]:
alt_prof

,P1,P2,P3
fiat,1.000000,0.800000,0.800000
vwPolo,0.700000,0.825000,0.715000
nissan,0.700000,0.800000,0.781250
toyota,0.700000,1.000000,0.990909
suzuki,0.633877,0.553471,0.815789
ford,0.378571,0.121429,0.537500


In [15]:
prof_alt

,fiat,vwPolo,nissan,toyota,suzuki,ford
P1,0.653846,0.442143,0.961538,0.300000,0.965031,1.000000
P2,0.463780,0.645669,1.000000,0.566929,0.785714,1.000000
P3,0.200000,0.500000,0.500000,0.500000,0.497417,0.763514
